In [ ]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_mistralai import MistralAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_mistralai.chat_models import ChatMistralAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [3]:
%pwd

'/workspaces/codespaces-blank/Source-Code-Analysis-Generative-AI/research'

In [4]:
!mkdir test_repo

mkdir: cannot create directory ‘test_repo’: File exists


In [ ]:
repo_path="repo_code/"

repo=Repo.clone_from("https://github.com/Harshitnitw/End-to-End-Medical-Chatbot-Generative-AI", to_path=repo_path)

In [6]:
loader=GenericLoader.from_filesystem(repo_path,
                                     glob="**/*",
                                     suffixes=[".py"],
                                     parser=LanguageParser(language=Language.PYTHON, parser_threshold=500))

In [7]:
documents=loader.load()

In [8]:
len(documents)

8

In [9]:
documents[0]

Document(metadata={'source': 'test_repo/setup.py', 'language': <Language.PYTHON: 'python'>}, page_content="from setuptools import find_packages, setup\n\nsetup(\n    name = 'Generative AI Project',\n    version= '0.0.0',\n    author= 'Harshit Kedia',\n    author_email= 'harshitkedia321@gmail.com',\n    packages= find_packages(),\n    install_requires = []\n\n)")

In [ ]:

document_splitter=RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON,
                                                               chunk_size=500,chunk_overlap=20)

In [11]:
texts=document_splitter.split_documents(documents)

In [12]:
texts

[Document(metadata={'source': 'test_repo/setup.py', 'language': <Language.PYTHON: 'python'>}, page_content="from setuptools import find_packages, setup\n\nsetup(\n    name = 'Generative AI Project',\n    version= '0.0.0',\n    author= 'Harshit Kedia',\n    author_email= 'harshitkedia321@gmail.com',\n    packages= find_packages(),\n    install_requires = []\n\n)"),
 Document(metadata={'source': 'test_repo/store_index.py', 'language': <Language.PYTHON: 'python'>}, page_content='from src.helper import load_pdf_file, text_split\nfrom pinecone import Pinecone, ServerlessSpec\nfrom langchain_mistralai import MistralAIEmbeddings\nfrom langchain_pinecone import PineconeVectorStore  \n\nimport os\nfrom dotenv import load_dotenv\n\nload_dotenv()\nos.environ["MISTRAL_API_KEY"] = os.getenv("MISTRAL_API_KEY")\npc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))\n\nembeddings = MistralAIEmbeddings(\n    model="mistral-embed",\n)\n\nindex_name="medical"'),
 Document(metadata={'source': 'test_repo/st

In [13]:
len(texts)

19

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
embeddings = MistralAIEmbeddings(
    model="mistral-embed",
)

/workspaces/codespaces-blank/Source-Code-Analysis-Generative-AI/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/workspaces/codespaces-blank/Source-Code-Analysis-Generative-AI/venv/lib/python3.10/site-packages/langchain_mistralai/embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [20]:
vectordb=Chroma.from_documents(texts,embedding=embeddings,persist_directory='database')

In [21]:
vectordb.persist()

/tmp/ipykernel_2082/3711397106.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [17]:
llm = ChatMistralAI(
    model="mistral-small-latest",
    temperature=0,
    max_retries=2,
    # other params...
)

In [18]:
memory=ConversationSummaryMemory(llm=llm,memory_key="chat_history", return_messages=True)

/tmp/ipykernel_22959/3310734444.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationSummaryMemory(llm=llm,memory_key="chat_history", return_messages=True)


In [23]:
qa=ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":8}),memory=memory)

In [19]:
question="What is make_request function?"

In [26]:
result=qa(question)

In [27]:
print(result['answer'])

The `make_request` function is defined to repeatedly invoke a retrieval-augmented generation (RAG) chain with a given question until it succeeds. Here is a breakdown of the function:

```python
def make_request(rag_chain, question, delay=1):
    while True:
        try:
            answer = rag_chain.invoke({"input": question})
            # Assuming the function returns the answer or processes it further
            return answer
        except Exception as e:
            # Handle the exception (e.g., log it, retry after a delay)
            logging.error(f"Error occurred: {e}")
            time.sleep(delay)
```

- **Parameters:**
  - `rag_chain`: The RAG chain object that will be used to process the question.
  - `question`: The question to be asked.
  - `delay`: The delay in seconds before retrying the request after an exception (default is 1 second).

- **Functionality:**
  - The function enters an infinite loop (`while True:`).
  - Inside the loop, it attempts to invoke the `rag_c